
Fiktives Datenaugmentierungs- & Trainings-Toolkit

Dieses Notebook ist **vollständig fiktiv** und dient als **strukturell analoges Beispiel** zu einem ML-Repository mit Zeitreihenklassifikation.  
Es zeigt eine plausible End-to-End-Pipeline von **README-ähnlicher Einführung** bis zu **Ergebnissen & Analyse** — jedoch mit **frei erfundenen Daten, Code und Inhalten**.

---

## Inhaltsverzeichnis

1. [README-ähnliche Einführung](#1)
2. [Datasets‑Beschreibung (fiktiv)](#2)
3. [Experimentelle Aufteilung](#3)
4. [Modellaufbau](#4)
5. [Training & Evaluation-Setup](#5)
6. [Ergebnisse & Analyse](#6)
7. [Anhang: Reproduzierbarkeit & Artefakte](#A)



<a id="1"></a>

## 1) README-ähnliche Einführung

**AstroPunch** simuliert Sensordaten eines fiktiven Mars-Rovers und demonstriert, wie man:
- ein **synthetisches Zeitreihen-Dataset** erzeugt,
- **Augmentierungen** für robuste Modelle anwendet,
- ein kleines **1D-CNN** in **PyTorch** trainiert,
- eine **Experimentkonfiguration** verwaltet,
- sowie **Metriken & Visualisierungen** erzeugt.

### 🎯 Ziele
- Zeige eine klare, reproduzierbare Struktur für ML-Experimente mit Zeitreihen.
- Mache die Pipeline (Daten → Modell → Training → Auswertung) nachvollziehbar.
- Bilde eine Vorlage ab, die man leicht an eigene Probleme anpassen kann.

### ⚡ Quickstart (lokal)
```bash
# (Optional) Neues Environment
python -m venv .venv && source .venv/bin/activate  # Windows: .venv\Scripts\activate

# Abhängigkeiten (Beispiel)
pip install numpy matplotlib scikit-learn torch torchvision torchaudio
```

> **Tipp:** GPU-Unterstützung für PyTorch ist optional. Das Notebook läuft auch auf CPU.



<a id="2"></a>

## 2) Datasets‑Beschreibung (fiktiv)

Wir simulieren **1D-Signale** (Länge `L=400` Samples) mit einer Abtastrate von `100 Hz`.  
Jede Zeitreihe gehört zu genau **einer** der folgenden **4 Klassen** (fiktive Rover-Betriebszustände):

- `CRUISE`: reguläre Fahrt — glatte Sinusmuster mit geringer Varianz  
- `DRILL`: Bohrbetrieb — überlagerte Periodik + sporadische Impulse  
- `SCAN`: Scan-Modus — modulierte Frequenzen (Chirp-ähnlich)  
- `FAULT`: Störung — unregelmäßige, verrauschte Muster

### 📦 Dateiformat & Struktur
- Für Illustration speichern wir Daten als `.npy`-Dateien mit einem `dict`:
  - `{"signal": np.ndarray[L], "label": str, "meta": dict}`
- Beispielhafte Ordner:
```
data/
 ├── train/
 ├── val/
 └── test/
```
- **Beispiel-Metadaten (`meta`)**:
  - `"id"`: eindeutige Sample-ID
  - `"sr"`: sample rate in Hz (hier: 100)
  - `"created"`: ISO-Zeitstempel der Generierung
  - `"generator"`: kurzer Hinweis zur Signal-Engine

> Das Dataset und alle Werte sind frei erfunden.


In [ ]:

# --- Datengenerierung (synthetisch & fiktiv) ---
from pathlib import Path
import numpy as np, json, time, math
rng = np.random.default_rng(7)

BASE = Path("data")
for split in ["train", "val", "test"]:
    (BASE / split).mkdir(parents=True, exist_ok=True)

SR = 100     # sample rate [Hz]
L  = 400     # signal length [samples]
CLASSES = ["CRUISE", "DRILL", "SCAN", "FAULT"]

def gen_cruise(L):
    t = np.linspace(0, L/SR, L, endpoint=False)
    f = rng.uniform(0.5, 1.5)
    sig = np.sin(2*np.pi*f*t) + rng.normal(0, 0.05, L)
    return sig

def gen_drill(L):
    t = np.linspace(0, L/SR, L, endpoint=False)
    f = rng.uniform(2.0, 4.0)
    base = 0.8*np.sin(2*np.pi*f*t)
    spikes = np.zeros(L)
    for _ in range(rng.integers(3, 7)):
        idx = rng.integers(0, L)
        spikes[idx:idx+3] += rng.uniform(1.5, 2.5)
    return base + spikes + rng.normal(0, 0.06, L)

def gen_scan(L):
    t = np.linspace(0, L/SR, L, endpoint=False)
    f0, f1 = rng.uniform(0.2, 0.6), rng.uniform(1.0, 2.0)
    phase = 2*np.pi*(f0*t + 0.5*(f1 - f0)*t**2/(t[-1] if t[-1] != 0 else 1))
    return np.sin(phase) + rng.normal(0, 0.05, L)

def gen_fault(L):
    # Unregelmäßig, verrauscht, mit gelegentlichen Plateaus
    sig = rng.normal(0, 0.3, L)
    for _ in range(rng.integers(2, 5)):
        a, b = sorted(rng.choice(np.arange(L), 2, replace=False))
        sig[a:b] += rng.uniform(-1.0, 1.0)
    return sig

GEN = {
    "CRUISE": gen_cruise,
    "DRILL":  gen_drill,
    "SCAN":   gen_scan,
    "FAULT":  gen_fault,
}

def save_sample(path, signal, label, i):
    meta = {
        "id": f"{label}_{i:05d}",
        "sr": SR,
        "created": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
        "generator": "AstroPunch v0 (fiktiv)"
    }
    np.save(path, {"signal": signal.astype(np.float32), "label": label, "meta": meta}, allow_pickle=True)

# Verteilungen pro Split (klein gehalten für Demo)
N_TRAIN, N_VAL, N_TEST = 600, 200, 200
per_class = lambda N: {c: N//len(CLASSES) for c in CLASSES}

def build_split(split, N):
    counts = per_class(N)
    i = 0
    for label, n in counts.items():
        for _ in range(n):
            sig = GEN[label](L)
            save_sample(BASE/split/f"{i:06d}.npy", sig, label, i)
            i += 1

for split, N in [("train", N_TRAIN), ("val", N_VAL), ("test", N_TEST)]:
    # Erzeuge nur, wenn Ordner leer ist
    if not any((BASE/split).iterdir()):
        build_split(split, N)

print("✓ Synthetic dataset ready at ./data (fiktiv)")



### 🔎 Schneller Blick in die Daten


In [ ]:

import numpy as np, matplotlib.pyplot as plt
from pathlib import Path

def load_any(split="train"):
    files = sorted((Path("data")/split).glob("*.npy"))
    arr = []
    for f in files[:8]:
        d = np.load(f, allow_pickle=True).item()
        arr.append(d)
    return arr

batch = load_any("train")

# 1 Plot pro Zelle (Regel), hier: erstes Beispiel
plt.plot(batch[0]["signal"])
plt.title(f"Beispielsignal (train[0]) — Label: {batch[0]['label']}")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.show()


In [ ]:

# Zweites Beispiel
plt.plot(batch[1]["signal"])
plt.title(f"Beispielsignal (train[1]) — Label: {batch[1]['label']}")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.show()



<a id="3"></a>

## 3) Experimentelle Aufteilung

### 📐 Splits
- **Train**: 600 Samples (≈ 70 %)
- **Val**: 200 Samples (≈ 20 %)
- **Test**: 200 Samples (≈ 10 %)

### 🧪 Metriken
- **Accuracy** (Top-1)
- **Confusion Matrix** (Test)
- Optional: **Precision/Recall/F1** pro Klasse (Bericht)

### 🔁 Augmentierung (online, nur im Training)
- **GaussianNoise**: Rauschen hinzufügen
- **TimeShift**: zyklisches Verschieben
- **Scale**: globale Amplitudenskalierung
- **RandomDropout1D**: sporadisches Nullsetzen einiger Punkte

### ♻️ Reproduzierbarkeit
- Fester Random-Seed für `numpy` und `torch`
- Fixierte Hyperparameter in einer zentralen **Config**


In [ ]:

import json, random, numpy as np, torch

CONFIG = {
    "seed": 2025,
    "data_dir": "data",
    "classes": ["CRUISE", "DRILL", "SCAN", "FAULT"],
    "sr": 100,
    "length": 400,
    "batch_size": 64,
    "num_workers": 0,
    "epochs": 8,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "dropout": 0.1,
    "augment": {
        "gaussian_noise_std": 0.05,
        "time_shift_max": 20,
        "scale_min": 0.9,
        "scale_max": 1.1,
        "dropout_prob": 0.02
    }
}

def set_all_seeds(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(CONFIG["seed"])
print("✓ Seeds gesetzt")


In [ ]:

from pathlib import Path
import numpy as np, torch
from torch.utils.data import Dataset, DataLoader

class GaussianNoise:
    def __init__(self, std=0.05): self.std = std
    def __call__(self, x):
        return x + np.random.normal(0, self.std, size=x.shape)

class TimeShift:
    def __init__(self, max_shift=20): self.max_shift = max_shift
    def __call__(self, x):
        k = np.random.randint(-self.max_shift, self.max_shift+1)
        return np.roll(x, k)

class Scale:
    def __init__(self, min_s=0.9, max_s=1.1): self.min_s, self.max_s = min_s, max_s
    def __call__(self, x):
        s = np.random.uniform(self.min_s, self.max_s)
        return x * s

class RandomDropout1D:
    def __init__(self, p=0.02): self.p = p
    def __call__(self, x):
        mask = np.random.rand(*x.shape) > self.p
        return x * mask

class Compose:
    def __init__(self, transforms): self.transforms = transforms
    def __call__(self, x):
        for t in self.transforms: x = t(x)
        return x

train_aug = Compose([
    GaussianNoise(CONFIG["augment"]["gaussian_noise_std"]),
    TimeShift(CONFIG["augment"]["time_shift_max"]),
    Scale(CONFIG["augment"]["scale_min"], CONFIG["augment"]["scale_max"]),
    RandomDropout1D(CONFIG["augment"]["dropout_prob"]),
])

class AstroDataset(Dataset):
    def __init__(self, root, split="train", classes=None, length=400, augment=None):
        self.root = Path(root)/split
        self.files = sorted(self.root.glob("*.npy"))
        self.classes = classes or CONFIG["classes"]
        self.cls2idx = {c:i for i,c in enumerate(self.classes)}
        self.length = length
        self.augment = augment if split=="train" else None

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        d = np.load(self.files[idx], allow_pickle=True).item()
        x = d["signal"].astype(np.float32)
        if x.shape[0] != self.length:
            # pad/trim to fixed length if nötig
            if x.shape[0] > self.length:
                x = x[:self.length]
            else:
                pad = self.length - x.shape[0]
                x = np.pad(x, (0,pad))
        if self.augment is not None:
            x = self.augment(x)
        y = self.cls2idx[d["label"]]
        # -> (C,L) für 1D-CNN
        return torch.from_numpy(x).unsqueeze(0), torch.tensor(y, dtype=torch.long)

train_ds = AstroDataset(CONFIG["data_dir"], "train", CONFIG["classes"], CONFIG["length"], augment=train_aug)
val_ds   = AstroDataset(CONFIG["data_dir"], "val",   CONFIG["classes"], CONFIG["length"], augment=None)
test_ds  = AstroDataset(CONFIG["data_dir"], "test",  CONFIG["classes"], CONFIG["length"], augment=None)

train_loader = DataLoader(train_ds, batch_size=CONFIG["batch_size"], shuffle=True,  num_workers=CONFIG["num_workers"])
val_loader   = DataLoader(val_ds,   batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
test_loader  = DataLoader(test_ds,  batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

len(train_ds), len(val_ds), len(test_ds)



<a id="4"></a>

## 4) Modellaufbau

Wir verwenden ein kleines **1D-CNN** mit:
- zwei Convolution-Blöcken (Conv1d → BatchNorm1d → ReLU → MaxPool1d),
- **Dropout** zur Regularisierung,
- einer **linearen** Klassifikationsschicht.

Die Architektur ist bewusst kompakt gehalten, damit das Notebook zügig läuft.


In [ ]:

import torch.nn as nn, torch

class ConvBlock(nn.Module):
    def __init__(self, c_in, c_out, k=5, p=2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(c_in, c_out, kernel_size=k, padding=k//2),
            nn.BatchNorm1d(c_out),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
    def forward(self, x): return self.block(x)

class AstroCNN(nn.Module):
    def __init__(self, n_classes=4, dropout=0.1, length=400):
        super().__init__()
        self.feat = nn.Sequential(
            ConvBlock(1, 16),
            ConvBlock(16, 32),
            nn.Dropout(dropout),
        )
        # Länge reduzierter Feature-Map ermitteln
        with torch.no_grad():
            dummy = torch.zeros(1,1,length)
            out = self.feat(dummy)
            feat_len = out.shape[-1] * out.shape[1]
        self.head = nn.Linear(feat_len, n_classes)

    def forward(self, x):
        z = self.feat(x)
        z = z.flatten(1)
        return self.head(z)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AstroCNN(n_classes=len(CONFIG["classes"]), dropout=CONFIG["dropout"], length=CONFIG["length"]).to(device)
model



<a id="5"></a>

## 5) Training & Evaluation-Setup

- **Optimierer:** Adam (`lr=1e-3`, `weight_decay=1e-4`)
- **Kriterium:** CrossEntropyLoss
- **Epochen:** 8 (für Demo; in echten Projekten erhöhen)
- **Auswahl des besten Modells:** bestes **Val-Accuracy**

Wir loggen **Loss** und **Accuracy** für Train/Val und visualisieren sie anschließend.


In [ ]:

from torch.optim import Adam
import torch.nn.functional as F
from collections import defaultdict
import numpy as np, torch, math, os

optimizer = Adam(model.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["weight_decay"])
criterion = nn.CrossEntropyLoss()

def run_epoch(loader, train: bool):
    model.train(mode=train)
    total, correct = 0, 0
    running_loss = 0.0
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        if train:
            optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        if train:
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        total += y.size(0)
        correct += (pred == y).sum().item()
    return running_loss/total, correct/total

history = defaultdict(list)
best_val, best_state = -1.0, None

for epoch in range(1, CONFIG["epochs"]+1):
    tr_loss, tr_acc = run_epoch(train_loader, True)
    va_loss, va_acc = run_epoch(val_loader, False)

    history["train_loss"].append(tr_loss)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(va_loss)
    history["val_acc"].append(va_acc)

    if va_acc > best_val:
        best_val = va_acc
        best_state = {k:v.cpu() for k,v in model.state_dict().items()}

    print(f"[{epoch:02d}/{CONFIG['epochs']}]  "
          f"train_loss={tr_loss:.4f}  train_acc={tr_acc:.3f}  |  "
          f"val_loss={va_loss:.4f}  val_acc={va_acc:.3f}")

# Lade bestes Val-Modell
if best_state is not None:
    model.load_state_dict(best_state)
    print(f"✓ Bestes Val-Accuracy: {best_val:.3f}")


In [ ]:

import matplotlib.pyplot as plt

plt.plot(history["train_loss"])
plt.plot(history["val_loss"])
plt.title("Loss über Epochen")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.legend(["Train", "Val"])
plt.show()


In [ ]:

plt.plot(history["train_acc"])
plt.plot(history["val_acc"])
plt.title("Accuracy über Epochen")
plt.xlabel("Epoche")
plt.ylabel("Accuracy")
plt.legend(["Train", "Val"])
plt.show()



<a id="6"></a>

## 6) Ergebnisse & Analyse

Wir bewerten das **beste** Validierungsmodell auf dem **Test**-Split und betrachten:
- Accuracy
- Confusion Matrix
- Klassifikationsbericht (Precision/Recall/F1)
- Beispielvorhersagen (Qualitativ)


In [ ]:

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np, torch

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for x,y in test_loader:
        x = x.to(device)
        logits = model(x)
        pred = logits.argmax(dim=1).cpu().numpy().tolist()
        y_pred.extend(pred)
        y_true.extend(y.numpy().tolist())

test_acc = (np.array(y_true) == np.array(y_pred)).mean()
print(f"Test-Accuracy: {test_acc:.3f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(CONFIG["classes"]))))
cm


In [ ]:

# Visualisierung der Confusion Matrix
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix (Test)")
plt.colorbar()
tick_marks = np.arange(len(CONFIG["classes"]))
plt.xticks(tick_marks, CONFIG["classes"], rotation=45)
plt.yticks(tick_marks, CONFIG["classes"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


In [ ]:

# Klassifikationsbericht (Text)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=CONFIG["classes"]))


In [ ]:

# Qualitative Beispiele: Plot einiger Test-Signale mit Vorhersage
import numpy as np, matplotlib.pyplot as plt, torch, random

indices = random.sample(range(len(test_ds)), k=min(4, len(test_ds)))
for idx in indices:
    x, y = test_ds[idx]
    with torch.no_grad():
        logits = model(x.unsqueeze(0).to(device))
        pred = int(logits.argmax(dim=1).cpu().item())
    plt.plot(x.squeeze(0).numpy())
    plt.title(f"True: {CONFIG['classes'][y]} | Pred: {CONFIG['classes'][pred]}")
    plt.xlabel("Samples")
    plt.ylabel("Amplitude")
    plt.show()



<a id="A"></a>

## Anhang: Reproduzierbarkeit & Artefakte

- **Config** & **Metriken** werden gespeichert.
- **Modellgewichte** (`.pt`) und Statistiken (`.json`) erleichtern spätere Auswertungen.
- Zusätzlich werden **Bibliotheksversionen** dokumentiert.


In [ ]:

from pathlib import Path
import json, pandas as pd, torch

out_dir = Path("artifacts")
out_dir.mkdir(parents=True, exist_ok=True)

# 1) Config speichern
with open(out_dir/"config.json", "w") as f:
    json.dump(CONFIG, f, indent=2)

# 2) Training-Historie als CSV
import pandas as pd
import numpy as np
hist_df = pd.DataFrame({
    "epoch": np.arange(1, len(history["train_loss"])+1),
    "train_loss": history["train_loss"],
    "train_acc": history["train_acc"],
    "val_loss": history["val_loss"],
    "val_acc": history["val_acc"]
})
hist_df.to_csv(out_dir/"history.csv", index=False)

# 3) Bestes Modell speichern
torch.save(model.state_dict(), out_dir/"best_model.pt")

# 4) Confusion Matrix & Test-Accuracy
np.save(out_dir/"confusion_matrix.npy", cm)
with open(out_dir/"test_metrics.json", "w") as f:
    json.dump({"test_accuracy": float((np.array(cm).trace()/np.array(cm).sum()) if cm.sum()>0 else 0.0)}, f, indent=2)

print("✓ Artefakte gespeichert in ./artifacts")


In [ ]:

# Bibliotheksversionen
import sys, numpy, sklearn, torch, matplotlib
print("Python :", sys.version.split()[0])
print("NumPy  :", numpy.__version__)
print("PyTorch:", torch.__version__)
print("sklearn:", sklearn.__version__)
print("mpl    :", matplotlib.__version__)



---

### ✅ Zusammenfassung
- Vollständige, strukturierte Pipeline (Daten → Augmentierung → Modell → Training → Auswertung).
- Alles frei erfunden und unabhängig von bestehenden Projekten.
- Kann als **Vorlage** für eigene Zeitreihen-Experimente dienen.

Viel Erfolg beim Anpassen! 🧑‍🚀
